In [1]:
import os 
# os.environ["WANDB_DISABLED"] = "true"
import pandas as pd
import torch
from tqdm import tqdm
import torch.nn.functional as F
from tqdm.auto import tqdm
from datetime import datetime
import wandb
import time
import os
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, PegasusConfig, get_cosine_schedule_with_warmup
from torch.optim import AdamW

from torch.utils.data import DataLoader, Dataset
from rouge import Rouge
from torch.optim.lr_scheduler import _LRScheduler
import math


2025-05-10 12:33:34.880318: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-10 12:33:35.523194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746855215.745438   74018 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746855215.810543   74018 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746855216.335558   74018 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
NUM_EPOCHS = 50
BATCH_SIZE = 4
FRAC_SAMPLE = 0.2
MAX_LENGTH_ARTICLE = 512
MIN_LENGTH_ARTICLE = 50
MAX_LENGTH_SUMMARY = 128
MIN_LENGTH_SUMMARY = 20
HIDDEN_DIM = 128
LEARNING_RATE = 1e-3
PATIENCE = 5  # For early stopping
WEIGHT_DECAY = 1e-4
NUM_CYCLES = 5


# model_dir = "../Model"
datafilter = "../dataft"
save_dir = "fine_tuned_pegasus_custom"
output_path = os.path.join(datafilter, "test_pred_5.csv")
os.makedirs(datafilter, exist_ok=True)
# os.makedirs(model_dir, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.makedirs(save_dir, exist_ok=True)

In [3]:
train_data = pd.read_csv("../dataset/train.csv")
validation_data = pd.read_csv("../dataset/validation.csv")
test_data = pd.read_csv("../dataset/test.csv")

# add col
train_data.rename(columns={"highlights": "summaries", "article":"articles"}, inplace=True)
validation_data.rename(columns={"highlights": "summaries","article":"articles"}, inplace=True)
test_data.rename(columns={"highlights": "summaries", "article":"articles"}, inplace=True)

train_data["article_word_count"] = train_data["articles"].astype(str).apply(lambda x: len(x.split()))
train_data["summary_word_count"] = train_data["summaries"].astype(str).apply(lambda x: len(x.split()))

validation_data["article_word_count"] = validation_data["articles"].astype(str).apply(lambda x: len(x.split()))
validation_data["summary_word_count"] = validation_data["summaries"].astype(str).apply(lambda x: len(x.split()))

test_data["article_word_count"] = test_data["articles"].astype(str).apply(lambda x: len(x.split()))
test_data["summary_word_count"] = test_data["summaries"].astype(str).apply(lambda x: len(x.split()))

# filter range
train_data = train_data[
    (train_data["article_word_count"] <= MAX_LENGTH_ARTICLE) & 
    (train_data["article_word_count"] >= MIN_LENGTH_ARTICLE) &
    (train_data["summary_word_count"] <= MAX_LENGTH_SUMMARY) &
    (train_data["summary_word_count"] >= MIN_LENGTH_SUMMARY)
]

validation_data = validation_data[
    (validation_data["article_word_count"] <= MAX_LENGTH_ARTICLE) & 
    (validation_data["article_word_count"] >= MIN_LENGTH_ARTICLE) &
    (validation_data["summary_word_count"] <= MAX_LENGTH_SUMMARY) &
    (validation_data["summary_word_count"] >= MIN_LENGTH_SUMMARY)
]
test_data = test_data[
    (test_data["article_word_count"] <= MAX_LENGTH_ARTICLE) & 
    (test_data["article_word_count"] >= MIN_LENGTH_ARTICLE) &
    (test_data["summary_word_count"] <= MAX_LENGTH_SUMMARY) &
    (test_data["summary_word_count"] >= MIN_LENGTH_SUMMARY)
]

train_sample = train_data.sample(frac=FRAC_SAMPLE, random_state=1)
validation_sample = validation_data.sample(frac=FRAC_SAMPLE, random_state=1)
test_sample = test_data.sample(frac=1, random_state=1)
train_sample.info()
print("\n")
validation_sample.info()
train_sample.to_csv(os.path.join(datafilter,"train_sample.csv"), index=False)
test_sample.to_csv(os.path.join(datafilter,"test_sample.csv"), index=False)
validation_sample.to_csv(os.path.join(datafilter,"validation_sample.csv"), index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 19198 entries, 144417 to 201560
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  19198 non-null  object
 1   articles            19198 non-null  object
 2   summaries           19198 non-null  object
 3   article_word_count  19198 non-null  int64 
 4   summary_word_count  19198 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 899.9+ KB


<class 'pandas.core.frame.DataFrame'>
Index: 994 entries, 8901 to 8365
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  994 non-null    object
 1   articles            994 non-null    object
 2   summaries           994 non-null    object
 3   article_word_count  994 non-null    int64 
 4   summary_word_count  994 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 46.6+ KB


In [4]:
train_sample = pd.read_csv("../dataft/train_sample.csv")
validation_sample = pd.read_csv("../dataft/validation_sample.csv")
test_sample = pd.read_csv("../dataft/test_sample.csv")
train_sample.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19198 entries, 0 to 19197
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  19198 non-null  object
 1   articles            19198 non-null  object
 2   summaries           19198 non-null  object
 3   article_word_count  19198 non-null  int64 
 4   summary_word_count  19198 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 750.1+ KB


In [5]:
# from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')
# model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(device)
# from transformers import PegasusTokenizer, PegasusForConditionalGeneration
# import torch


# # Hiển thị tổng số tham số và trainable parameters
# total_params = sum(p.numel() for p in model.parameters())
# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f"🔢 Total parameters: {total_params:,}")
# print(f"🧠 Trainable parameters: {trainable_params:,}\n")

from transformers import PegasusConfig, PegasusForConditionalGeneration
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')
config = PegasusConfig(
    vocab_size=96103,  # bắt buộc khớp tokenizer
    max_position_embeddings=512,
    encoder_layers=6,           # giảm so với 16 (pegasus-large)
    decoder_layers=6,
    encoder_attention_heads=8,
    decoder_attention_heads=8,
    d_model=512,
    d_ff=2048,
    dropout=0.1,
    attention_dropout=0.1,
    activation_dropout=0.1,
    init_std=0.02,
    scale_embedding=True,
    use_cache=True,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

model = PegasusForConditionalGeneration(config)
model.resize_token_embeddings(len(tokenizer))  # quan trọng nếu tokenizer được tùy biến
model.to(device)


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 512, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 512, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=512, bias=True)
          (final_layer_norm): La

In [6]:
class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=MAX_LENGTH_ARTICLE, max_output_length=MAX_LENGTH_SUMMARY):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        article = self.data.iloc[index]["articles"]
        summary = self.data.iloc[index]["summaries"]
        input_ids = self.tokenizer.encode(article, max_length=self.max_input_length, truncation=True, padding="max_length")
        output_ids = self.tokenizer.encode(summary, max_length=self.max_output_length, truncation=True, padding="max_length")
        return {"input_ids": input_ids, "attention_mask": [int(token_id != 0) for token_id in input_ids], "decoder_input_ids": output_ids[:-1], "decoder_attention_mask": [1] * (len(output_ids) - 1), "labels": output_ids[1:]}
train_df = train_sample
test_df = test_sample
val_df = validation_sample

In [7]:
train_dataset = SummarizationDataset(train_df, tokenizer)
val_dataset = SummarizationDataset(val_df, tokenizer)
test_dataset = SummarizationDataset(test_df, tokenizer)

In [8]:
def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    decoder_input_ids = [item["decoder_input_ids"] for item in batch]
    decoder_attention_mask = [item["decoder_attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]
    max_input_length = max(len(ids) for ids in input_ids)
    max_output_length = max(len(ids) for ids in decoder_input_ids)
    input_ids = [ids + [0] * (max_input_length - len(ids)) for ids in input_ids]
    attention_mask = [mask + [0] * (max_input_length - len(mask)) for mask in attention_mask]
    decoder_input_ids = [ids + [0] * (max_output_length - len(ids)) for ids in decoder_input_ids]
    decoder_attention_mask = [mask + [0] * (max_output_length - len(mask)) for mask in decoder_attention_mask]
    labels = [ids + [-100] * (max_output_length - len(ids)) for ids in labels]
    
    labels = torch.tensor(labels)
    labels[labels == tokenizer.pad_token_id] = -100
    
    return {"input_ids": torch.tensor(input_ids), "attention_mask": torch.tensor(attention_mask), "decoder_input_ids": torch.tensor(decoder_input_ids), "decoder_attention_mask": torch.tensor(decoder_attention_mask), "labels": torch.tensor(labels)}
# def collate_fn(batch):
#     input_ids = [item["input_ids"] for item in batch]
#     attention_mask = [item["attention_mask"] for item in batch]
#     labels = [item["labels"] for item in batch]

#     max_input_length = max(len(ids) for ids in input_ids)
#     max_output_length = max(len(lab) for lab in labels)

#     def pad(seq, max_len, pad_value):
#         return seq + [pad_value] * (max_len - len(seq))

#     input_ids = torch.tensor([pad(ids, max_input_length, tokenizer.pad_token_id) for ids in input_ids])
#     attention_mask = torch.tensor([pad(mask, max_input_length, 0) for mask in attention_mask])
#     labels = torch.tensor([pad(lab, max_output_length, -100) for lab in labels])

#     return {
#         "input_ids": input_ids,
#         "attention_mask": attention_mask,
#         "labels": labels
#     }


In [9]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,collate_fn=collate_fn)

In [10]:
# no_decay = ['bias', 'LayerNorm.weight']    # cần ít regularization hơn weight
# optimizer_grouped_parameters = [
#     {
#         'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#         'weight_decay': WEIGHT_DECAY,
#     },
#     {
#         'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
#         'weight_decay': 0.0,
#     },
# ]
# # optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)

# # Scheduler
# num_training_steps = len(train_loader) * NUM_EPOCHS
# # scheduler = get_cosine_schedule_with_warmup(
# #     optimizer,
# #     num_warmup_steps=int(0.2 * num_training_steps),
# #     num_training_steps=num_training_steps,
# #     num_cycles=NUM_CYCLES
# # )

optimizer = AdamW(model.parameters(), lr=5e-5)
# Scheduler
num_training_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.2 * num_training_steps),
    num_training_steps=num_training_steps,
    num_cycles=NUM_CYCLES
)

In [11]:
# # Add custom scheduler class (place this at the top of your script)
# class CosineWarmupWithBounds(_LRScheduler):
#     def __init__(self, optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, max_lr=1e-4, min_lr=1e-5, last_epoch=-1):
#         self.num_warmup_steps = num_warmup_steps
#         self.num_training_steps = num_training_steps
#         self.num_cycles = num_cycles
#         self.max_lr = max_lr
#         self.min_lr = min_lr
#         super(CosineWarmupWithBounds, self).__init__(optimizer, last_epoch)

#     def get_lr(self):
#         step = self.last_epoch + 1
#         if step < self.num_warmup_steps:
#             lr = self.min_lr + (self.max_lr - self.min_lr) * step / self.num_warmup_steps
#         else:
#             progress = (step - self.num_warmup_steps) / (self.num_training_steps - self.num_warmup_steps)
#             cosine_factor = 0.5 * (1.0 + math.cos(math.pi * progress * self.num_cycles))
#             lr = self.min_lr + (self.max_lr - self.min_lr) * cosine_factor
#         lr = max(self.min_lr, min(self.max_lr, lr))
#         return [lr for _ in self.optimizer.param_groups]

# # Update optimizer and scheduler
# optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)
# scheduler = CosineWarmupWithBounds(
#     optimizer,
#     num_warmup_steps=int(0.2 * num_training_steps),
#     num_training_steps=num_training_steps,
#     num_cycles=NUM_CYCLES,
#     max_lr=1e-4,
#     min_lr=1e-5
# )

In [12]:
# Weights & Biases initialization
wandb.init(
    project="Finetune-Summarization",
    name=f"pegasus-custom-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    config={
        "model": "Pegasus_custom",
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "weight_decay": WEIGHT_DECAY,
        "num_epochs": NUM_EPOCHS,
        "num_cycles": NUM_CYCLES,
        "data_ratio": FRAC_SAMPLE,
        "warm_up": "Cosine",
    }
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vubkk67 (vubkk67-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin


In [13]:
# Save best model and early stopping
best_val_loss = float("inf")
epochs_no_improve = 0
wandb.watch(model)

# Training loop
for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        if torch.isnan(loss):
            print("⚠️ Detected NaN loss. Skipping batch.")
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # <--- CLIP
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            val_loss += outputs.loss.item()
        val_loss /= len(val_loader)
        current_lr = scheduler.get_last_lr()[0]

    # W&B log
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "lr": current_lr,
        "best_val_loss": best_val_loss
    })

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save_pretrained(save_dir)
        tokenizer.save_pretrained(save_dir)
        print(f"Saved best model to `{save_dir}` at epoch {epoch+1}")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Early stopping
    if epochs_no_improve >= PATIENCE:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

    print(
        f"Epoch {epoch+1:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"LR: {current_lr:.6f} | "
        f"Time: {time.time() - start_time:.2f}s"
    )

# W&B end
wandb.finish()


Epoch 1 [Train]:   0%|          | 0/4800 [00:00<?, ?it/s]

/tmp/ipykernel_74018/1907396855.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {"input_ids": torch.tensor(input_ids), "attention_mask": torch.tensor(attention_mask), "decoder_input_ids": torch.tensor(decoder_input_ids), "decoder_attention_mask": torch.tensor(decoder_attention_mask), "labels": torch.tensor(labels)}


Epoch 1 [Val]:   0%|          | 0/249 [00:00<?, ?it/s]

Saved best model to `fine_tuned_pegasus_custom` at epoch 1
Epoch 01 | Train Loss: 8.9396 | Val Loss: 7.1121 | LR: 0.000005 | Time: 1672.77s


Epoch 2 [Train]:   0%|          | 0/4800 [00:00<?, ?it/s]

...

Epoch 16 | Train Loss: 2.5207 | Val Loss: 4.7192 | LR: 0.000001 | Time: 1846.88s


Epoch 17 [Train]:   0%|          | 0/4800 [00:00<?, ?it/s]

Epoch 17 [Val]:   0%|          | 0/249 [00:00<?, ?it/s]

Early stopping triggered at epoch 17


best_val_loss,█▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁
epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
lr,▂▂▃▄▄▅▆▇▇██▇▅▃▂▁▁
train_loss,█▆▅▅▄▄▄▄▃▃▃▂▂▁▁▁▁
val_loss,█▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁
best_val_loss,4.5965
epoch,17
lr,0.0
train_loss,2.46188
val_loss,4.72354


In [14]:
# tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
# model = PegasusForConditionalGeneration.from_pretrained(save_dir).to(device)

In [15]:
# Evaluation
model.to(device)
model.eval()

predictions = []

with torch.no_grad():
    for step, batch in enumerate(tqdm(test_loader, desc="Generating summaries")):
        batch = {k: v.to(device) for k, v in batch.items()}
        output_ids = model.generate(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=MAX_LENGTH_SUMMARY,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True
        )
        batch_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions.extend(batch_preds)

test_sample = test_df.iloc[:len(predictions)].copy()
test_sample["predicted_summary"] = predictions
test_sample.to_csv(output_path, index=False)

print(f"✅ File has been saved at: {output_path}")

Generating summaries:   0%|          | 0/1056 [00:00<?, ?it/s]

/tmp/ipykernel_74018/1907396855.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {"input_ids": torch.tensor(input_ids), "attention_mask": torch.tensor(attention_mask), "decoder_input_ids": torch.tensor(decoder_input_ids), "decoder_attention_mask": torch.tensor(decoder_attention_mask), "labels": torch.tensor(labels)}


✅ File has been saved at: ../dataft/test_pred_5.csv


In [16]:
test_pred = pd.read_csv(output_path)

In [17]:
display(test_pred[["articles","summaries", "predicted_summary"]].head(10))

,articles,summaries,predicted_summary
0,A Florida bus passenger was arrested for throw...,"Joel Parker, 33, was riding the bus in St John...","d Joel Parker, 33, and threw a bar at St. Park..."
1,Aston Villa may be able to sign Cordoba strike...,Aston Villa have held talks over Cordoba strik...,first-one could sign for £2.5million this seas...
2,A South Carolina mother of four died over the ...,"Adam Leheup ran Fitness 535 in Columbia, South...","ing, a mother of a mother-of-four and her brot..."
3,A playground in Tokyo has been found to have d...,Soil underneath a slide in the park showed ext...,ing from high levels of high levels of radiati...
4,A lonely shepherd has been found dead alongsid...,"Body of Jose Alberto, 58, discovered at home i...","s were found dead in San Josep, 58, two young ..."
5,"Four years after signing for Arsenal, Wellingt...",Wellington Silva signed for Arsenal in 2011 fo...,ing has been granted his passport for £3.5mill...
6,Joey Barton has urged QPR to use their win ove...,Queens Park Rangers strode to a 4-1 victory ag...,ing's side beat QPR 4-1 over West Brom . The 1...
7,An Indian woman who holds several awards for t...,Smita Srivastava currently holds record for lo...,one of the best hair for her hair . She is the...
8,Drilling threes on the buzzer is all good as f...,LeBron James posted an unhappy picture at the ...,ing James James is one of the Cleveland Bulls ...
9,A teenager killed by police in Illinois on Sat...,"Justus Howell, 17, was running from scene of a...",. Howell revealed he was shot twice for one sh...


In [18]:
# Tính điểm ROUGE
if "summaries" in test_pred.columns:
    rouge = Rouge()
    scores = rouge.get_scores(predictions, test_sample["summaries"].tolist(), avg=True)

    print("ROUGE scores:")
    print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
    print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
    print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")
else:
    print("⚠️ Không tìm thấy cột 'summaries' để tính ROUGE.")

ROUGE scores:
ROUGE-1: 0.2858
ROUGE-2: 0.0646
ROUGE-L: 0.2656
